In [309]:
msr = {}
with open('data/clf/msr_positive_pairs.txt') as f:
    for t in f.readlines():
        r, n1, n2 = t.split()
        msr[(r, n1)] = n2

files = ['rlog/small_sample_for_recall.txt_leave_file_list.out',
        'rlog/small_sample_for_recall.txt_leave_code.out',
        'rlog/small_sample_for_recall.txt_leave_text.out',
        'rlog/small_sample_for_recall.txt_leave_location.out',
        'rlog/small_sample_for_recall.txt_leave_pattern.out',
        'rlog/small_sample_for_recall.txt_new.out']

def check(file):
    # threshold = 0.6257216989

    num, tot = 0, 0

    num2, num3 = 0, 0

    w = []
    with open(file) as f:
        for t in f.readlines():
            tot += 1

            ts = t.split(':')
            r, n2 = ts[0].strip().split()
            op = ts[1].strip()[2:-2].split('), (')

            n1, v = op[0].split(',')

            v = float(v)

            w.append(v)

            if v < threshold:
                continue


            if n1 == msr.get((r, n2), ''):
                num2 += 1

            for obj in op[:5]:
                n1, v = obj.split(',')
                if n1 == msr.get((r, n2), ''):
                    num3 += 1
                    break

            num += 1

    print(num, tot, 1.0 * num / tot)

    #print(file, 1.0 * num2 / tot, num2, tot)
    # print(1.0 * num3 / tot, num3, tot)

    #print(len(w), '->', 50)
    w = sorted(w, reverse=True)
    # print(file, ':', w[39])
    tw = file.split('_')[-1].split('.out')[0]
    if tw == 'list':
        tw = 'file_list'
    tw = '\'' + tw + '\''
    print(tw, ':', (w[30 - 1] + w[31 - 1]) / 2, ',')
    
'''
for file in files:
    check(file)
'''

check(files[3])


29 200 0.145
'location' : 0.6220857158077546 ,


In [340]:
from git import *

way_select = ['file_list', 'code', 'text', 'location', 'pattern', 'new']


def work(way):
    file = 'rlog/sample_small_data.txt_leave_%s.out' % way

    if way == 'new':
        file = 'rlog/sample_small_data.txt_new.out'

    outfile = 'rlog/webpage_%s.html' % way
    with open(outfile, 'w') as outf:
        pass

    
    threshold_select =  { #% 20%
    'file_list' : 0.6081185134217753,
    'code' : 0.5815402870741628,
    'text' : 0.6065632181226785,
    'location' : 0.6069270229613258,
    'pattern' : 0.5994309119726756,
    'new' : 0.6023691773882736,
    }
    '''
    threshold_select =  { # 15%
    'file_list' : 0.6233914080341519 ,
    'code' : 0.5891530735030057 ,
    'text' : 0.6440541699084077 ,
    'location' : 0.6233203523695693 ,
    'pattern' : 0.6173069060241599 ,
    'new' : 0.6140082744041935 ,
    }
    '''
    
    p_tot = 0
    p_num = 0

    tot = 0
    threshold = threshold_select[way]
        
    label = {}
    with open('rlog/labeled.txt') as f:
        for t in f.readlines():
            r, n2, n1, v, status = t.split()
            if 'Unknown' in status:
                continue
            label[(r, n2, n1)] = status

    with open(file) as f:
        for t in f.readlines():
            tot += 1

            ts = t.split(':')
            r, n2 = ts[0].strip().split()
            op = ts[1].strip()[2:-2].split('), (')

            if (r, n2) == ('angular/angular.js', '14068'):
                continue
            
            
            if r == 'ceph/ceph':
                continue
            if r == 'dotnet/corefx':
                continue
            
            
            n1, v = op[0].split(',')


            v = float(v)
    
            if v >= threshold:
                p_tot += 1

                if (r, n2, n1) in label:
                    st = label[(r, n2, n1)]
                else:
                    st = 'Unknown'
                    if (n2 in get_another_pull(get_pull(r, n1))) or (n1 in get_another_pull(get_pull(r, n2))):
                        st += '(mention)'

                print(r, n2, n1, v, st, sep='\t')
                
                if 'Unknown' in st:
                    # raise Exception('lack label for', r, n2, n1)
                    print('lack label for', r, n2, n1, sep='\t')
                    # print(r, n2, n1, v, st, sep='\t')
                    pass
                
                if 'Y' in st:
                    p_num += 1

                if 'mention' in st:
                    p_num += 1

                p2 = get_pull(r, n2)
                p1 = get_pull(r, n1)
                s2 = '<a href="https://github.com/%s/pull/%s" target="_blank"> %s </a>' % (r, n2, r + '/' + n2 + ':' + p2['title'])
                s1 = '<a href="https://github.com/%s/pull/%s" target="_blank"> %s </a>' % (r, n1, r + '/' + n1 + ':' + p1['title'])
                
                with open(outfile, 'a', encoding="utf-8") as outf:
                    print(s2, '----', s1, '<hr>', file=outf)

    print(way, 1.0 * p_num / p_tot, p_num, p_tot, tot, sep='\t')
    # print(way, 1.0 * p_num / p_tot, sep='\t')

'''
for way in way_select:
    # print('-----%s-----' % way)
    work(way)
'''

work('location')

docker/docker	2657	2460	0.6100675318927571	Y
joomla/joomla-cms	9140	6964	0.6209938766303842	N(?)
joomla/joomla-cms	20424	19603	0.6575884167026796	Y
hashicorp/terraform	7528	7418	0.6118985432611699	N
hashicorp/terraform	7584	7456	0.614425520741566	N
hashicorp/terraform	10436	9398	0.6153352836618532	Y
hashicorp/terraform	13065	12940	0.6470377140231438	Y
django/django	3909	3694	0.6520253475341083	Y
django/django	4655	4651	0.6612464174196263	Y
django/django	5662	2585	0.7079815512230344	Y
cocos2d/cocos2d-x	3398	3385	0.6269488072441247	N
elastic/elasticsearch	18106	17864	0.6145270908683481	N
location	0.5833333333333334	7	12	668
